<a href="https://colab.research.google.com/github/abhaskumarsinha/Scaling-Down-Transformers-Investigating-Emergent-Phenomena-in-Tiny-Models/blob/main/Experiment%203.2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://www.github.com/abhaskumarsinha/Corpus2GPT.git

fatal: destination path 'Corpus2GPT' already exists and is not an empty directory.


In [2]:
!pip install keras==3.6

In [3]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
import os

os.environ["KERAS_BACKEND"] = "tensorflow"

import numpy as np
import keras

from Corpus2GPT.utils import LanguagePairDataset as LPD
from Corpus2GPT.tokenizer import SPM_Tokenizer
from Corpus2GPT.models import build_GPT
from Corpus2GPT.training_utils.distribution_utils import *

from Corpus2GPT.inference import Generative_inference

In [5]:
total_vocabs = 12
total_chars = 1

sentence_len = 4
total_instances = 2500
instance_split = '|'
sentence_split = '.'

tokenizer_vocab_size = 54

gpt_input_len = 45
embedding_dim = 900
num_decoder = 3
dropout_rate = 0
num_heads = 30
head_dims = 30
fc_dim_factor = 5
epsilon = 1e-6

epochs = 100
batch_size = 256

In [6]:
dataset = LPD()

dataset.valid_chars = dataset.valid_chars + list("0123456789!@#$%^&*()_+{}")

dataset.create_vocabs(total_vocabs, total_chars) # Language A
dataset.create_vocabs(total_vocabs, total_chars) # Language B
dataset.create_vocabs(total_vocabs, total_chars) # Language C
dataset.create_vocabs(total_vocabs, total_chars) # Language D

A2B, B2A = dataset.create_map(0, 1)
A2C, C2A = dataset.create_map(0, 2)
A2D, D2A = dataset.create_map(0, 3)



# D -> A -> B
D2B = {d: A2B[a] for d, a in D2A.items()}
# B -> A -> D
B2D = {b: A2D[a] for b, a in B2A.items()}

# D -> A -> C
D2C = {d: A2C[a] for d, a in D2A.items()}
# C -> A -> D
C2D = {c: A2D[a] for c, a in C2A.items()}

# B -> A -> C (B to A to C)
B2C = {b: A2C[a] for b, a in B2A.items()}  # B -> A -> C
# C -> B (C to A to B)
C2B = {c: A2B[a] for c, a in C2A.items()}  # C -> A -> B


dataset.write_dataset_to_file(A2B, B2A, sentence_len, 5000, './test_translation_dataset.txt', append=False, instance_split=instance_split, sentence_split=sentence_split)
dataset.write_dataset_to_file(A2D, D2A, sentence_len, 4500, './test_translation_dataset.txt', append=True, instance_split=instance_split, sentence_split=sentence_split)
dataset.write_dataset_to_file(A2C, C2A, sentence_len, 4000, './test_translation_dataset.txt', append=True, instance_split=instance_split, sentence_split=sentence_split)
dataset.write_dataset_to_file(D2B, B2D, sentence_len, 100, './test_translation_dataset.txt', append=True, instance_split=instance_split, sentence_split=sentence_split)
dataset.write_dataset_to_file(D2C, C2D, sentence_len, 200, './test_translation_dataset.txt', append=True, instance_split=instance_split, sentence_split=sentence_split)
dataset.write_dataset_to_file(B2C, C2B, sentence_len, 300, './test_translation_dataset.txt', append=True, instance_split=instance_split, sentence_split=sentence_split)

In [7]:
SPM_Tokenizer(vocab_size = tokenizer_vocab_size, corpus='./test_translation_dataset.txt')

In [8]:
distribute_scope = get_distribution_scope("tpu")
with distribute_scope():
    tokenizer = SPM_Tokenizer(vocab_model_file='tokenizer_.model', input_size = gpt_input_len+1)

    for (X, Y) in tokenizer.load_dataset(['./test_translation_dataset.txt']):
        X, Y = np.array(X), np.array(Y)

    optimizer = keras.optimizers.Adam(learning_rate=0.0001, clipnorm=1)
    GPT, flops = build_GPT(gpt_input_len, tokenizer_vocab_size, embedding_dim, num_decoder, dropout_rate, num_heads, head_dims, fc_dim_factor, epsilon, optimizer=optimizer)
    GPT.summary()

    history = GPT.fit(X, Y, batch_size=1024, epochs=200)

Success!


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ token_and_position_embedding         │ (None, 45, 900)             │          89,100 │
│ (TokenAndPositionEmbedding)          │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ decoder (Decoder)                    │ (None, 45, 900)             │      11,352,600 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ decoder_1 (Decoder)                  │ (None, 45, 900)             │      11,352,600 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ decoder_2 (Decoder)                  │ (None, 45, 900)             │      11,352,600 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_18 (Dense)                     │ (None, 45, 55)              │          49,555 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 34,196,455 (130.45 MB)

 Trainable params: 34,196,455 (130.45 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 56s 5s/step - loss: 4.7245
Epoch 2/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 2s 201ms/step - loss: 2.1325
Epoch 3/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 171ms/step - loss: 1.9246
Epoch 4/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 2s 173ms/step - loss: 1.8654
Epoch 5/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 2s 234ms/step - loss: 1.8223
Epoch 6/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 2s 272ms/step - loss: 1.7931
Epoch 7/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 2s 248ms/step - loss: 1.7368
Epoch 8/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 2s 290ms/step - loss: 1.6521
Epoch 9/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 2s 250ms/step - loss: 1.5382
Epoch 10/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 2s 272ms/step - loss: 1.4395
Epoch 11/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 2s 292ms/step - loss: 1.3086
Epoch 12/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 2s 274ms/step - loss: 1.2062
Epoch 13/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 170ms/step - loss: 1.1461
Epoch 14/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 170ms/step - loss: 1.0892
Epoch 15/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 2s 254ms/step - loss: 1.0522
Epoch 

In [9]:
inference = Generative_inference(GPT, tokenizer, input_len=gpt_input_len)
# Set num_test to 100
num_test = 100

# Testing for A2B
score = 0
for i in range(num_test):
    score += dataset.evaluate_model_accuracy(A2B, B2A, sentence_len, inference, 8, 7, num_prompts=6) / num_test

print('\n   Total Hits (A2B): ' + str(score) + ' ')

# Testing for A2D
score = 0
for i in range(num_test):
    score += dataset.evaluate_model_accuracy(A2D, D2A, sentence_len, inference, 8, 7, num_prompts=6) / num_test

print('   Total Hits (A2D): ' + str(score) + ' ')

# Testing for A2C
score = 0
for i in range(num_test):
    score += dataset.evaluate_model_accuracy(A2C, C2A, sentence_len, inference, 8, 7, num_prompts=6) / num_test

print('   Total Hits (A2C): ' + str(score) + ' ')

# Testing for D2B
score = 0
for i in range(num_test):
    score += dataset.evaluate_model_accuracy(D2B, B2D, sentence_len, inference, 8, 7, num_prompts=6) / num_test

print('   Total Hits (D2B): ' + str(score) + ' ')

# Testing for D2C
score = 0
for i in range(num_test):
    score += dataset.evaluate_model_accuracy(D2C, C2D, sentence_len, inference, 8, 7, num_prompts=6) / num_test

print('   Total Hits (D2C): ' + str(score) + ' ')

# Testing for B2C
score = 0
for i in range(num_test):
    score += dataset.evaluate_model_accuracy(B2C, C2B, sentence_len, inference, 8, 7, num_prompts=6) / num_test

print('   Total Hits (B2C): ' + str(score) + ' ')



   Total Hits (A2B): 1.0000000000000007 
   Total Hits (A2D): 1.0000000000000007 
   Total Hits (A2C): 1.0000000000000007 
   Total Hits (D2B): 0.935 
   Total Hits (D2C): 0.9975000000000007 
   Total Hits (B2C): 0.9925000000000007 
